In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pytpc
from pytpc.constants import *
import pytpc.simulation
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import yaml
from pytpc.utilities import find_exclusion_region, read_lookup_table
import math
from effsim.paramgen import uniform_param_generator
from effsim.effsim import EventSimulator

In [ ]:
with open('/home/chen/Real Data/config_e15503a_runs_92-104.yml', 'r') as f:
    config = yaml.load(f)

mcfitter = pytpc.fitting.MCFitter(config)

In [ ]:
tilt = config['tilt'] * degrees 
#clock = config['clock']
#beam_en = config['beam_enu0']
padplane = mcfitter.padplane
efield = np.array(config['efield'])
bfield = np.array(config['bfield'])
#tmat = pytpc.utilities.tilt_matrix(-tilt)
# efield_uvw = tmat.dot(efield)
# bfield_uvw = tmat.dot(bfield)
mass_num = config['mass_num']
charge_num = config['charge_num']
gas = pytpc.gases.InterpolatedGas(config['gas_name'], config['gas_pressure'])
beam_enu0 = config['beam_enu0']
beam_charge = config['beam_charge']
beam_mass = config['beam_mass']
max_en = config['tracker_max_en']
vd = np.array(config['vd'])
max_beam_angle = (config['max_beam_angle']*math.pi)/180
beam_origin_z = config['beam_origin_z']
#beampads = np.fromfile(config['beampads_path'], sep=',', dtype='int')

In [ ]:
sim = EventSimulator(config)

In [ ]:
num_evts = 100
pgen = uniform_param_generator(beam_enu0, beam_mass, beam_charge, mass_num, max_beam_angle, beam_origin_z, gas, num_evts)
p = next(pgen)
print(p)

In [ ]:
evtRandom, ctr = sim.make_event(p[0][0], p[0][1], p[0][2], p[0][3], p[0][4], p[0][5])
pyevtRandom = sim.convert_event(evtRandom)
Random_xyzs = pyevtRandom.xyzs(peaks_only=True, return_pads=True, baseline_correction=True, cg_times=True)

In [ ]:
Random_xy = Random_xyzs[:, 0:2]
Random_xy_C = np.ascontiguousarray(Random_xy, dtype=np.double)
Random_cx, Random_cy = pytpc.cleaning.hough_circle(Random_xy_C)
#The center of curvature of the track, in mm.

Random_uvw, (Random_cu, Random_cv) = mcfitter.preprocess(Random_xyzs[:, 0:5], center=(Random_cx, Random_cy), rotate_pads=True)
Random_uvw_values = Random_uvw.values
print(Random_cu, Random_cv)

Random_mcres, Random_minChis, Random_all_params, Random_good_param_idx = mcfitter.process_event(Random_uvw, Random_cu, Random_cv, return_details=True)
print(Random_mcres)

In [ ]:
evtClean, ctr = sim.make_event(Random_mcres['x0'], Random_mcres['y0'], Random_mcres['z0'], Random_mcres['enu0'], Random_mcres['azi0'], Random_mcres['pol0'])
#evtClean, ctr = sim.make_event(p[0][0], p[0][1], p[0][2], p[0][3], p[0][4], p[0][5])

pyevtClean = sim.convert_event(evtClean)
sim_xyzs = pyevtClean.xyzs(peaks_only=True,   return_pads=True, baseline_correction=True, cg_times=True)

In [ ]:
plt.plot(Random_xyzs[:, 2], Random_xyzs[:, 0], 'r.')
plt.plot(sim_xyzs[:, 2], sim_xyzs[:, 0], 'b.')
plt.show()
plt.plot(Random_xyzs[:, 2], Random_xyzs[:, 1], 'r.')
plt.plot(sim_xyzs[:, 2], sim_xyzs[:, 1], 'b.')
plt.show()
plt.plot(Random_xyzs[:, 0], Random_xyzs[:, 1], 'r.')
plt.plot(sim_xyzs[:, 0], sim_xyzs[:, 1], 'b.')
plt.show()



fig = plt.figure(figsize=(20, 14))

ax = fig.add_subplot(2, 3, 1, projection='3d') #2,3,1 meaning the plot will take the 1st position on a grid with 2 rows and three columns
ax.scatter(Random_xyzs[:,0],Random_xyzs[:,1],Random_xyzs[:,2])

ax2 = fig.add_subplot(2, 3, 2, projection='3d')
ax2.scatter(sim_xyzs[:,0],sim_xyzs[:,1],sim_xyzs[:,2])